## Get Data Set ##

In [18]:
# instalación de librerias
%pip install sec-edgar-api
%pip install finnhub-python
%pip install yfinance

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\jpbla\appdata\local\programs\python\python311\lib\site-packages\fasttext-0.9.2-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\jpbla\appdata\local\programs\python\python311\lib\site-packages\fasttext-0.9.2-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\jpbla\appdata\local\programs\python\python311\lib\site-packages\fasttext-0.9.2-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [6]:
#importación de librerias necesarias
import pandas as pd
import finnhub
from tqdm import tqdm
import yfinance as yf
import os
import json
import time
from datetime import datetime
from sec_edgar_api import EdgarClient
from dateutil.relativedelta import relativedelta

## Descripción de Bases de Datos ##
- EDGAR  - (Electronic Data Gathering, Analysis, and Retrieval ) base de datos pública del gobierno de Estados Unidos donde se  suben los estados financieros de todas las empresas publicas.

- Finnhub - es un repositorio en linea de información financiera, es accesible a través de un API.

- YF - es un repositorio del movimiento diario del stock de las empresas.

In [7]:
#configuraciones necesarias
edgar = EdgarClient(user_agent="tec a01793474@tec.mx")
finnhub_client = finnhub.Client(api_key='coopla9r01qtvljfpqvgcoopla9r01qtvljfpr00')

In [63]:
def get_data(data_dir,com_list):
  """ Función para obtener y generar archivos json con la información"""
  data = []
  os.makedirs(f'{data_dir}', exist_ok=True) 
  os.makedirs(f'{data_dir}/news', exist_ok=True) 
  os.makedirs(f'{data_dir}/submissions', exist_ok=True) 
  os.makedirs(f'{data_dir}/facts', exist_ok=True) 
  os.makedirs(f'{data_dir}/stock', exist_ok=True)
  start_date = (datetime.today() + relativedelta(months=-12)).strftime('%Y-%m-%d')
  end_date = datetime.today().strftime('%Y-%m-%d')
  
  for i in tqdm(range(0,len(com_list)-1)):
    symbol = com_list['symbol'][i]
    cipk = com_list['cipk'][i]
    
    try:
      
      weekly_news = finnhub_client.company_news(symbol, _from=start_date, to=end_date)
      weekly_news = [{
                        "date": datetime.fromtimestamp(n['datetime']).strftime('%Y%m%d%H%M%S'),
                        "headline": n['headline'],
                        "summary": n['summary'],
                    } for n in weekly_news
                ]
      weekly_news.sort(key=lambda x: x['date'])
        
      json_file1 = f'{data_dir}/news/{symbol}_{start_date}_{end_date}_news.json' 
        
      with open(json_file1, mode='w', newline='',encoding="utf-8") as file1:
       file1.write(json.dumps({'news':weekly_news}))
        
      submissions = edgar.get_submissions(cik=cipk)
        
      json_file2 = f'{data_dir}/submissions/{symbol}_submissions.json' 
        
      with open(json_file2, mode='w', newline='',encoding="utf-8") as file2:
        file2.write(json.dumps({'submissions':submissions}))
              
      facts = edgar.get_company_facts(cik=cipk)
        
      json_file3 = f'{data_dir}/facts/{symbol}_facts.json' 
        
      with open(json_file3, mode='w', newline='',encoding="utf-8") as file3:
        file3.write(json.dumps({'facts':facts}))
        
      stock = yf.download(symbol, start=start_date, end=end_date).reset_index()
        
      json_file4 = f'{data_dir}/stock/{symbol}_stock.json' 
        
      stock.to_json(json_file4)
                  
      data.append({'symbol':symbol,
                      'data': {
                        'sumbissions': submissions,
                        'facts':facts,
                        'news':weekly_news
                      }
                  })
      
      time.sleep(1)
      
    except Exception as e:
      print(f" {e} in symbol {symbol}")
      
  return data

## Datos a Obtener ##
- Facts - información de la empresa y documentos financieros subidos
- News - noticias relevantes de la empresa
- Stock - movimientos del stock de la empresa
- Submissions - documentos subidos

In [64]:
#listado de empresas 
com_list = pd.read_csv('./data/ticker_select.txt', sep='\t',names=['symbol', 'cipk']) 

In [65]:
#obtener y generar archivos
data = get_data('data',com_list)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

 FinnhubAPIException(status_code: 403): You don't have access to this resource. in symbol MC.PA


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
 24%|██▍       | 32/133 [01:36<05:47,  3.44s/it]

 FinnhubAPIException(status_code: 403): You don't have access to this resource. in symbol NESN.SW


[*********************100%%**********************]  1 of 1 completed
 27%|██▋       | 36/133 [01:41<02:37,  1.62s/it]

...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
 89%|████████▊ | 118/133 [05:44<00:54,  3.63s/it]

 FinnhubAPIException(status_code: 403): You don't have access to this resource. in symbol ALV.DE


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
 98%|█████████▊| 131/133 [06:17<00:04,  2.10s/it]

 FinnhubAPIException(status_code: 403): You don't have access to this resource. in symbol AI.PA


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
100%|██████████| 133/133 [06:23<00:00,  2.88s/it]


### Preguntas sobre los datos ###
- ¿Hay valores faltantes en el conjunto de datos? ¿Se pueden identificar patrones de ausencia? 
Si existen algunas llamadas a las cuales no tenemos permisos o no se encuentran datos, en este caso se omiten.

- ¿Cuáles son las estadísticas resumidas del conjunto de datos?
Siendo 4 tipos de docuementos se deben separar en cada uno.
    - Facts - tiene datos de la empresa que nos sirven para obtener mayor información.
    - Submissions - tienen información financiera que podemos obtener información de estados.
    - Stock - es la información diaria del movimiento de las acciones aun no estamos seguros si podemos utilizarla pero seria interesante.
    - News - este es el prinicipal dato que utilizaremos, sirve para poder hacer las relaciones de los datos y de donde realizar las preguntas.

- ¿Hay valores atípicos en el conjunto de datos?
No, todos los valores son estandares el unico problema que hemos visto es en las noticias no estan bien los caracteres, eso podemos filtrarlo más adelante.

- ¿Cuál es la cardinalidad de las variables categóricas?
En este caso las variables categóricas se definen de diferent forma para nuestro proyecto en este caso deben ser atributos o nodos y relaciones pero esto lo definirá el LLM.

- ¿Existen distribuciones sesgadas en el conjunto de datos? ¿Necesitamos aplicar alguna transformación no lineal?
Si pero esto es esperado, no es necesaria la transformación.

- ¿Se identifican tendencias temporales? (En caso de que el conjunto incluya una dimensión de tiempo).
Si hay tendencias sobre todo en los datos de stock pero no es seguro que utilicemos esos datos, lo que si podemos hacer es un analisis de aumentar o disminuir el stock lo cual haria una variable interesante.

- ¿Hay correlación entre las variables dependientes e independientes?
En este caso, eso es lo que estamos buscando las relaciones entre los datos para generar le grafo de conocimiento.

- ¿Cómo se distribuyen los datos en función de diferentes categorías?
Cada uno de los documentos tiene un proposito diferente y por lo tanto no hay distribución. Estas categorias o relaciones se tienen que obtener de manera automatica.

- ¿Existen patrones o agrupaciones (clusters) en los datos con características similares?
Este es el tema del cual tenemos que trabajar que se determinen las relaciones de manera automatica.

- ¿Se deberían normalizar las imágenes para visualizarlas mejor?
No es necesario. Se necesitan los datos tal cual se obtuvieron.

- ¿Hay desequilibrio en las clases de la variable objetivo?
Nuestra variable objetivo es la relación entre los datos.

In [66]:
# Ejemplos
facts = pd.read_json('data/facts/AAPL_facts.json', encoding = 'utf-8')
news = pd.read_json('data/news/AAPL_2023-05-04_2024-05-04_news.json', encoding = 'utf-8')
stock = pd.read_json('data/stock/AAPL_stock.json', encoding = 'utf-8')
sub = pd.read_json('data/submissions/AAPL_submissions.json', encoding = 'utf-8')

In [87]:
facts_data = pd.DataFrame(dict(facts['facts'][2]))
facts_data.head()

C:\Users\jpbla\AppData\Local\Temp\ipykernel_5852\3277446292.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  facts_data = pd.DataFrame(dict(facts['facts'][2]))


,dei,us-gaap
EntityCommonStockSharesOutstanding,"{'label': 'Entity Common Stock, Shares Outstan...",NaN
EntityPublicFloat,"{'label': 'Entity Public Float', 'description'...",NaN
AccountsPayable,NaN,{'label': 'Accounts Payable (Deprecated 2009-0...
AccountsPayableCurrent,NaN,"{'label': 'Accounts Payable, Current', 'descri..."
AccountsReceivableNetCurrent,NaN,"{'label': 'Accounts Receivable, after Allowanc..."


In [89]:
news.head()

,news
0,"{'date': '20240501170359', 'headline': 'Dow Jo..."
1,"{'date': '20240501193000', 'headline': 'Apple,..."
2,"{'date': '20240501194100', 'headline': 'Invest..."
3,"{'date': '20240501204000', 'headline': 'How Mu..."
4,"{'date': '20240501204631', 'headline': 'Sell A..."


In [90]:
stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-05-04,164.889999,167.039993,164.309998,165.789993,164.909760,81235400
1,2023-05-05,170.979996,174.300003,170.759995,173.570007,172.648468,113316400
2,2023-05-08,172.479996,173.850006,172.110001,173.500000,172.578827,55962800
3,2023-05-09,173.050003,173.539993,171.600006,171.770004,170.858017,45326900
4,2023-05-10,173.020004,174.029999,171.899994,173.559998,172.638504,53724500


In [92]:
sub.head()

,submissions
addresses,"{'mailing': {'street1': 'ONE APPLE PARK WAY', ..."
category,Large accelerated filer
cik,320193
description,
ein,942404110
